In [1]:
import pandas as pd
import numpy as np

import glob
import tqdm
import gc

print('>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!')
f'pd : {pd.__version__}  |  np : {np.__version__}  |  tqdm : {tqdm.__version__}'

>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!


'pd : 1.0.5  |  np : 1.16.5  |  tqdm : 4.47.0'

## 셀-행정동 매핑자료 로드

In [2]:
region_name_map = pd.read_csv('../2_ADMDONG_CD-ADMDONG_NM.csv')
region_name_map = region_name_map[region_name_map.sido_nm == '제주특별자치도'].\
                    reset_index(drop=True).drop(columns=['sido_cd', 'sido_nm'])

display(region_name_map.head(3))
print('제주도 행정동 수 :', region_name_map.shape[0])
print(f'제주도 행정동 코드 (상위 5개) : {region_name_map.admdong_cd.tolist()[:5]} ...')

,sgg_cd,sgg_nm,admdong_cd,admdong_nm
0,50110,제주시,50110250,한림읍
1,50110,제주시,50110520,일도2동
2,50110,제주시,50110530,이도1동


제주도 행정동 수 : 43
제주도 행정동 코드 (상위 5개) : [50110250, 50110520, 50110530, 50110560, 50110580] ...


In [3]:
cell_code_map = pd.read_csv('../1_50CELL-ADMDONG_CD.csv', usecols=['id', 'admdong_cd'])
cell_code_map = cell_code_map[cell_code_map.admdong_cd.isin(region_name_map.admdong_cd)]

display(cell_code_map.head(3))
print('제주도 소속 50셀 코드 수 :', cell_code_map.shape[0])
print(f'제주도 소속 50셀 코드 (상위 5개) : {cell_code_map.id.tolist()[:5]} ...')

,id,admdong_cd
0,30509513,50110310
1,30509514,50110310
2,30521701,50110310


제주도 소속 50셀 코드 수 : 747189
제주도 소속 50셀 코드 (상위 5개) : [30509513, 30509514, 30521701, 30521702, 30521703] ...


## 행정동별 월별 유동인구 총합 계산 - 시간대별, 성연령별 (1월만)

In [4]:
%%time

filename = '../JEJU_201901.csv'
year = int(filename[8:12])
month = int(filename[12:14])

tmp_df = pd.read_csv(filename,
                     usecols=['id', 'timezn_cd', 'm00', 'm10', 'm15', 'm20', 'm25', 'm30',
                              'm35', 'm40', 'm45', 'm50', 'm55', 'm60', 'm65', 'm70', 'f00', 'f10',
                              'f15', 'f20', 'f25', 'f30', 'f35', 'f40', 'f45', 'f50', 'f55', 'f60',
                              'f65', 'f70', 'total'])

tmp_time_df = tmp_df[['id', 'timezn_cd', 'total']].merge(cell_code_map, how='left', on='id').\
                groupby(['admdong_cd', 'timezn_cd'])['total'].sum().reset_index()
tmp_sexAge_df = tmp_df.drop(columns=['timezn_cd', 'total']).merge(cell_code_map, how='left', on='id').\
                    groupby('admdong_cd')[['m00', 'm10', 'm15', 'm20', 'm25', 'm30', 'm35', 'm40', 'm45', 'm50',
                                           'm55', 'm60', 'm65', 'm70', 'f00', 'f10', 'f15', 'f20', 'f25', 'f30',
                                           'f35', 'f40', 'f45', 'f50', 'f55', 'f60', 'f65', 'f70']].sum().reset_index()
tmp_sexAge_df = pd.melt(tmp_sexAge_df, id_vars=['admdong_cd'], var_name='sex_age', value_name='total')
del tmp_df

tmp_time_df['year'] = year
tmp_time_df['month'] = month
tmp_time_df = region_name_map.merge(tmp_time_df[['admdong_cd', 'year', 'month', 'timezn_cd', 'total']],
                                    how='left', on='admdong_cd')

tmp_sexAge_df['year'] = year
tmp_sexAge_df['month'] = month
tmp_sexAge_df = region_name_map.merge(tmp_sexAge_df[['admdong_cd', 'year', 'month', 'sex_age', 'total']],
                                      how='left', on='admdong_cd')


print(tmp_time_df.shape)
display(tmp_time_df.head(3))

print('\n', tmp_sexAge_df.shape)
display(tmp_sexAge_df.head(3))

(1032, 8)


,sgg_cd,sgg_nm,admdong_cd,admdong_nm,year,month,timezn_cd,total
0,50110,제주시,50110250,한림읍,2019,1,0,33086
1,50110,제주시,50110250,한림읍,2019,1,1,25067
2,50110,제주시,50110250,한림읍,2019,1,2,20057



 (1204, 8)


,sgg_cd,sgg_nm,admdong_cd,admdong_nm,year,month,sex_age,total
0,50110,제주시,50110250,한림읍,2019,1,m00,414
1,50110,제주시,50110250,한림읍,2019,1,m10,13659
2,50110,제주시,50110250,한림읍,2019,1,m15,93933


Wall time: 1min


## 행정동별 월별 유동인구 총합 계산 - 시간대별, 성연령별  (19년 1월 ~ 20년 4월 - 16개월 간)

In [5]:
%%time

time_result_list = []
sexAge_result_list = []

for filename in tqdm.tqdm_notebook(glob.glob('../JEJU_20*.csv')):
    year = int(filename[8:12])
    month = int(filename[12:14])

    tmp_df = pd.read_csv(filename,
                         usecols=['id', 'timezn_cd', 'm00', 'm10', 'm15', 'm20', 'm25', 'm30',
                                  'm35', 'm40', 'm45', 'm50', 'm55', 'm60', 'm65', 'm70', 'f00', 'f10',
                                  'f15', 'f20', 'f25', 'f30', 'f35', 'f40', 'f45', 'f50', 'f55', 'f60',
                                  'f65', 'f70', 'total'])

    tmp_time_df = tmp_df[['id', 'timezn_cd', 'total']].merge(cell_code_map, how='left', on='id').\
                    groupby(['admdong_cd', 'timezn_cd'])['total'].sum().reset_index()
    tmp_sexAge_df = tmp_df.drop(columns=['timezn_cd', 'total']).merge(cell_code_map, how='left', on='id').\
                        groupby('admdong_cd')[['m00', 'm10', 'm15', 'm20', 'm25', 'm30', 'm35', 'm40', 'm45', 'm50',
                                               'm55', 'm60', 'm65', 'm70', 'f00', 'f10', 'f15', 'f20', 'f25', 'f30',
                                               'f35', 'f40', 'f45', 'f50', 'f55', 'f60', 'f65', 'f70']].sum().reset_index()
    tmp_sexAge_df = pd.melt(tmp_sexAge_df, id_vars=['admdong_cd'], var_name='sex_age', value_name='total')
    del tmp_df

    tmp_time_df['year'] = year
    tmp_time_df['month'] = month
    tmp_time_df = region_name_map.merge(tmp_time_df[['admdong_cd', 'year', 'month', 'timezn_cd', 'total']],
                                        how='left', on='admdong_cd')
    time_result_list.append(tmp_time_df)
    

    tmp_sexAge_df['year'] = year
    tmp_sexAge_df['month'] = month
    tmp_sexAge_df = region_name_map.merge(tmp_sexAge_df[['admdong_cd', 'year', 'month', 'sex_age', 'total']],
                                          how='left', on='admdong_cd')
    sexAge_result_list.append(tmp_sexAge_df)
    
    
    gc.collect()


print(len(time_result_list), len(sexAge_result_list))

C:\Users\dslab01\anaconda3\envs\gis_prac\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



16 16
Wall time: 20min 20s


## 최종 데이터프레임 결측치 확인 및 저장

In [6]:
time_result_df = pd.concat(time_result_list, ignore_index=True)
display(time_result_df)
display(time_result_df.isna().sum())

time_result_df.to_parquet('../ETL_source_output/제주도_월별_행정동별_시간대별_총합.parquet')

,sgg_cd,sgg_nm,admdong_cd,admdong_nm,year,month,timezn_cd,total
0,50110,제주시,50110250,한림읍,2019,1,0,33086
1,50110,제주시,50110250,한림읍,2019,1,1,25067
2,50110,제주시,50110250,한림읍,2019,1,2,20057
3,50110,제주시,50110250,한림읍,2019,1,3,18365
4,50110,제주시,50110250,한림읍,2019,1,4,17390
...,...,...,...,...,...,...,...,...
16507,50130,서귀포시,50130610,중문동,2020,4,19,152315
16508,50130,서귀포시,50130610,중문동,2020,4,20,114996
16509,50130,서귀포시,50130610,중문동,2020,4,21,86585
16510,50130,서귀포시,50130610,중문동,2020,4,22,59048


sgg_cd        0
sgg_nm        0
admdong_cd    0
admdong_nm    0
year          0
month         0
timezn_cd     0
total         0
dtype: int64

In [7]:
sexAge_result_df = pd.concat(sexAge_result_list, ignore_index=True)
display(sexAge_result_df)
display(sexAge_result_df.isna().sum())

sexAge_result_df.to_parquet('../ETL_source_output/제주도_월별_행정동별_성연령별_총합.parquet')

,sgg_cd,sgg_nm,admdong_cd,admdong_nm,year,month,sex_age,total
0,50110,제주시,50110250,한림읍,2019,1,m00,414
1,50110,제주시,50110250,한림읍,2019,1,m10,13659
2,50110,제주시,50110250,한림읍,2019,1,m15,93933
3,50110,제주시,50110250,한림읍,2019,1,m20,141344
4,50110,제주시,50110250,한림읍,2019,1,m25,214588
...,...,...,...,...,...,...,...,...
19259,50130,서귀포시,50130610,중문동,2020,4,f50,150554
19260,50130,서귀포시,50130610,중문동,2020,4,f55,124412
19261,50130,서귀포시,50130610,중문동,2020,4,f60,69596
19262,50130,서귀포시,50130610,중문동,2020,4,f65,21685


sgg_cd        0
sgg_nm        0
admdong_cd    0
admdong_nm    0
year          0
month         0
sex_age       0
total         0
dtype: int64